# Running MD on a system of butane and water

It's time to run your first MD simulation! In the code that follows below, you'll load in the force field parameters, initial coordinates (3D structure), and topology for a box of 500 water molecules and 50 butane molecules. Then you'll subject this system to energy minimization, followed by a short equilibration simulation and a longer production simulation (for calculating statistics). All of this is facilitated by a Python library called OpenMM (see: https://openmm.org).

In [ ]:
from openmm import app
import openmm as mm
from simtk import unit
from sys import stdout
import time as time

In the code below, see if you can identify the lines of code that provide the following ingredients necessary for MD:

* initial coordinates
* topology
* force field
* integrator
* thermostat
* barostat

Note that in some cases, one line of code may do more than one thing!

In [ ]:
prmtop = app.AmberPrmtopFile('butane-water.prmtop')
inpcrd = app.AmberInpcrdFile('butane-water.inpcrd')

system = prmtop.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=0.9*unit.nanometer, constraints=app.HBonds)
temperature = 298.15*unit.kelvin
pressure = 1*unit.bar
integrator = mm.LangevinIntegrator(temperature, 1/unit.picosecond, 2*unit.femtoseconds)
system.addForce(mm.MonteCarloBarostat(pressure, temperature))
# run the simulations using CPUs
#platform = mm.Platform.getPlatformByName('CPU')
# run the system using an NVIDIA GPU
platform = mm.Platform.getPlatformByName('CUDA')
simulation = app.Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)

The code below subjects the system to 100 steps of energy minimization. This is necessary because the molecules are packed into the simulation box in random orientations and some of the interatomic distances are likely too short (compared to what they should be). This leads to very large potential energies and very strong forces that can render any MD simulation numerically unstable. Energy minimization "relieves" these issues and ensures that the simulation starts from a more stable configuration.

In [ ]:
st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy before minimization is", st.getPotentialEnergy())

print('Minimizing...')
simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy after minimization is", st.getPotentialEnergy())

After energy minimization, we typically run a short simulation to bring the system to its equilibrium temperature and density.

In [ ]:
simulation.context.setVelocitiesToTemperature(298.15*unit.kelvin)
print('Equilibrating...')
tinit=time.time()
simulation.step(10000)
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Finally, we run the production simulation that we will later visualize (to see what happens!) and perform data anaylsis on. 

Note that we write the atomic coordinates out to a **trajectory file** (in this case: `butane-water_sim.dcd`) every 250 time steps. We don't do this every simulation step because otherwise the file size would become prohibitively large.

This simulation is 500,000 time steps long. If $\Delta t$ for each time step is 2 femtoseconds (1 fs = 10$^{-15}$ s), how long is this simulation in *pico*seconds (1 ps = 10$^{-12}$ s)?

In [ ]:
simulation.reporters.append(app.DCDReporter('butane-water_sim.dcd', 250))
simulation.reporters.append(app.StateDataReporter(stdout, 10000, step=True, time=True,
    potentialEnergy=True, temperature=True, density=True, speed=True, separator='\t'))


print('Running Production...')
simulation.step(500000)
print('Done!')

**Congratulations!** You've just performed your first MD simulation. Now let's take a look at your simulation trajectory!

# Visualizing the butane-water MD simulation

After running an MD simulation, it's often useful (and fun!) to visualize the simulation trajectory. To do this we will read in the trajectory file (using the library MDTraj) and then view it in this notebook (using the library NGLview).

In [ ]:
import mdtraj as md
import nglview as nv

The code below is how we load in the trajectory file. We'll also gather the atom names and topology into pandas DataFrames.

In [ ]:
traj = md.load('butane-water_sim.dcd', top='butane-water.prmtop')
atoms, bonds = traj.topology.to_dataframe()
atoms

Next, we visualize the trajectory using NGLview. Note that we can use different **representations** for different molecules (or even atoms) when performing our visualization.

In [ ]:
view = nv.show_mdtraj(traj)
view.clear_representations()
view.add_spacefill('BUT')
view.add_ball_and_stick('HOH')
view

# Your turn #1: what's happening here?

What do you see happening to the butane molecules and water molecules as the simulation progresses? Is there something about the respective chemical properties of these molecules that could explain this?

# Your turn #2: repeat the MD simulation and visualization workflow with another molecular system

Carry out the following procedure:

1. Make a copy of this notebook.
2. Replace every instance of `butane` with `ethylenediamine` in the new notebook. (Don't forget the notebook title/filename!)
3. Execute the MD portion of the notebook to completion. 
4. After the MD portion has finished, execute the visualization portion. (There is an additional change you'll need to make in the visualization portion; see if you can figure it out on your own!) 
5. Try to answer **Your turn #1** again for ethylenediamine.